In [ ]:
#|default_exp api

In [ ]:
#|export
from datetime import datetime
def ping(): return f"Kernel OK at {datetime.now():%H:%M:%S}"
ping()

'Kernel OK at 19:29:22'

In [ ]:
#|export
import httpx, pydantic

class Point(pydantic.BaseModel):
    lat: float
    lon: float
    def ll(self): return f"{self.lon},{self.lat}"


In [ ]:
#|export
import asyncio, httpx

async def geocode(query:str) -> Point:
    """Return the first OSM match for the search string."""
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": query, "format": "json", "limit": 1}
    async with httpx.AsyncClient() as cx:
        r = await cx.get(url, params=params, timeout=10)
    j = r.json()[0]
    return Point(lat=float(j["lat"]), lon=float(j["lon"]))


In [ ]:
#|export
OSRM_URL = "https://router.project-osrm.org/route/v1"

async def osrm_car(src: Point, dest: Point):
    url = f"{OSRM_URL}/car/{src.ll()};{dest.ll()}"
    async with httpx.AsyncClient() as cx:
        r = await cx.get(url, params={"overview": "false"}, timeout=10)
    r.raise_for_status()
    return r.json()["routes"][0]["duration"]        # seconds



In [ ]:
#|export
import os, httpx

ORS_URL = "https://api.openrouteservice.org/v2/directions"

async def ors_bike(src: Point, dst: Point, *, profile: str = "cycling-regular") -> float:
    """
    Return cycling travel time *in minutes* between two points using OpenRouteService.
    `profile` can be: cycling-regular, cycling-road, cycling-mountain, cycling-electric.
    """
    api_key = os.getenv("ORS_KEY")
    if not api_key:
        raise RuntimeError("Set ORS_KEY env-var with your OpenRouteService API token")

    url = f"{ORS_URL}/{profile}"
    body = {"coordinates": [[src.lon, src.lat], [dst.lon, dst.lat]]}

    async with httpx.AsyncClient() as cx:
        r = await cx.post(url, json=body, headers={"Authorization": api_key}, timeout=15)
        r.raise_for_status()

    seconds = r.json()["features"][0]["properties"]["summary"]["duration"]
    return round(seconds / 60, 1)           # minutes with one decimal


In [ ]:
#|export
from typing import List, Literal
from dataclasses import dataclass

Mode = Literal["car", "bike", "pt"]

@dataclass
class Step:
    instruction: str
    distance_m: float
    duration_s: float

@dataclass
class Route:
    mode: Mode
    distance_m: float
    duration_s: float
    geometry: List[Point]      # decoded polyline as list of Points
    steps:    List[Step]       # optional turn-by-turn


In [ ]:
#|export
async def osrm_car(src: Point, dst: Point) -> Route:
    url = f"{OSRM_URL}/car/{src.ll()};{dst.ll()}"
    params = {"overview": "full", "steps": "true", "geometries": "polyline"}
    async with httpx.AsyncClient() as cx:
        r = await cx.get(url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()["routes"][0]

    coords = [Point(lat=lat, lon=lon) for lon, lat in polyline.decode(data["geometry"])]

    def _instr(stp):
        m = stp["maneuver"]
        parts = [m["type"], m.get("modifier"), stp.get("name")]
        return " ".join(p for p in parts if p)

    steps = [
        Step(_instr(s), s["distance"], s["duration"])
        for leg in data["legs"] for s in leg["steps"]
    ]

    return Route(
        mode="car",
        distance_m=data["distance"],
        duration_s=data["duration"],
        geometry=coords,
        steps=steps,
    )


In [ ]:
#|export
import os, httpx, polyline

async def ors_bike(src: Point, dst: Point, profile: str = "cycling-regular") -> Route:
    api_key = os.getenv("ORS_KEY")
    if not api_key:
        raise RuntimeError("ORS_KEY missing")

    url  = f"{ORS_URL}/{profile}"
    body = {
        "coordinates": [[src.lon, src.lat], [dst.lon, dst.lat]],
        "instructions": True,
        "geometry": True,              # returns an encoded polyline
    }
    headers = {"Authorization": api_key}

    async with httpx.AsyncClient() as cx:
        r = await cx.post(url, json=body, headers=headers, timeout=15)
    r.raise_for_status()

    feat  = r.json()["features"][0]
    props = feat["properties"]["summary"]

    # decode the polyline string to Point objects
    coords = [Point(lat=lat, lon=lon) for lon, lat in polyline.decode(feat["geometry"])]

    steps  = [
        Step(step["text"], step["distance"], step["duration"])
        for step in feat["properties"]["segments"][0]["steps"]
    ]

    return Route(
        mode        = "bike",
        distance_m  = props["distance"],
        duration_s  = props["duration"],
        geometry    = coords,
        steps       = steps,
    )



In [ ]:
#|export
import os, json, httpx, polyline
from typing import List

# Digitransit v2 router (HSL region).  Change to "finland", "waltti", … if needed.
_DT_URL = "https://api.digitransit.fi/routing/v2/hsl/gtfs/v1"
HEADERS = {
    "digitransit-subscription-key": os.getenv("DIGITRANSIT_KEY", "ee36f515bc2d426fa25325dd51734266"), 
    "Content-Type": "application/json"
}

# GraphQL query – fastest single itinerary
_PLAN_Q = """
query Plan($orig: PlanLabeledLocationInput!, $dest: PlanLabeledLocationInput!) {
  planConnection(origin: $orig, destination: $dest, first: 1) {
    edges {
      node {
        duration
        walkDistance
        legs {
          mode distance duration
          from { name lat lon }
          to   { name lat lon }
          legGeometry { points }
        }
      }
    }
  }
}"""

async def digitransit_pt(
    src: Point,
    dst: Point,
    *,
    api_key: str | None = None,
    dataset: str = "hsl",
) -> Route:
    """
    Fetch the fastest **public-transport itinerary** between *src* and *dst*
    from Digitransit Routing v2 (GTFS-v1).

    Returns
    -------
    Route
        mode="pt", total distance/time, decoded geometry, per-leg `Step`s.
    """
   

    url = _DT_URL.replace("hsl", dataset)      # quick dataset switch

    variables = {
        "orig": {"location": {"coordinate": {"latitude": src.lat, "longitude": src.lon}}},
        "dest": {"location": {"coordinate": {"latitude": dst.lat, "longitude": dst.lon}}},
    }

    async with httpx.AsyncClient() as cx:
        r = await cx.post(
            url,
            json={"query": _PLAN_Q, "variables": variables},
            headers=HEADERS,
            timeout=20,
        )
    r.raise_for_status()
    body = r.json()
    if "errors" in body:
        raise RuntimeError(body["errors"][0]["message"])

    node = body["data"]["planConnection"]["edges"][0]["node"]

    # --- build geometry & steps ------------------------------------------------
    geom: List[Point] = []
    steps: List[Step] = []
    for leg in node["legs"]:
        pts = [Point(lat=lat, lon=lon)
               for lon, lat in polyline.decode(leg["legGeometry"]["points"])]
        if geom and pts:
            pts = pts[1:]                     # avoid duplicate joint
        geom.extend(pts)

        instr = f"{leg['mode'].title()} {leg['from']['name']} → {leg['to']['name']}"
        steps.append(Step(instr, leg["distance"], leg["duration"]))

    return Route(
        mode="pt",
        distance_m=node["walkDistance"] + sum(l["distance"] for l in node["legs"] if l["mode"] != "WALK"),
        duration_s=node["duration"],
        geometry=geom,
        steps=steps,
    )


In [ ]:
#|export
import httpx, datetime as dt

HSL_PR_BASE = "https://p.hsl.fi/api/v1"          # Park-&-Ride API
DEFAULT_FACILITY = 285            # Urheilupuisto 2025-05 feed

async def hsl_parking_status(facility_id: int = DEFAULT_FACILITY) -> dict:

    """
    Live vacancy for a Park-&-Ride facility (default **Urheilupuisto = 117**).

    Returns a dict with keys:
    * capacity            – total spaces
    * availableCapacity   – free spaces now
    * occupiedSpaces      – in use now
    * timestamp           – ISO UTC string
    """
    url = f"{HSL_PR_BASE}/facilities/{facility_id}/status.json"
    async with httpx.AsyncClient() as cx:
        r = await cx.get(url, timeout=10)
    r.raise_for_status()
    j = r.json()

    # normalise timestamp to UTC
    ts = dt.datetime.fromisoformat(j["timestamp"]).astimezone(dt.timezone.utc)
    j["timestamp"] = ts.isoformat()
    return j


In [ ]:
#|export

import os, httpx, datetime as dt
from typing import List, Optional
from pydantic import BaseModel, Field

_FT = "https://parking.fintraffic.fi/api/v1"
_DEFAULT_APP_ID = "datatalks-bot"

# ──────────────────────────────────────────────────────────────
# Pydantic models (only the fields we really use)
# ──────────────────────────────────────────────────────────────
class UtilRow(BaseModel):
    facilityId: int
    capacityType: str
    usage: str
    timestamp: str
    spacesAvailable: int
    capacity: int
    openNow: bool

class Facility(BaseModel):
    id: int
    name: dict                              # {fi:.., sv:.., en:..}
    operatorId: Optional[int] = None
    builtCapacity: dict[str, int] = Field(default_factory=dict)
    status: str

from pydantic import RootModel

class EnumList(RootModel[List[str]]):        
    pass


# ──────────────────────────────────────────────────────────────
# Fintraffic Parking API client
# ──────────────────────────────────────────────────────────────
class FintrafficParking:
    """
    Tiny convenience wrapper for the Fintraffic (LIIPI) Parking REST API.
    """

    def __init__(
        self,
        *,
        token: str | None = None,
        app_id: str = _DEFAULT_APP_ID,
        timeout: float = 10.0,
    ):
        """
        Parameters
        ----------
        token   : str | None
            Bearer token for **mutable** endpoints (PUT).  Leave None for read-only access.
        app_id  : str
            Value for the `Digitraffic-User` header (3-20 chars, a-z, 0-9, _-. /)
        timeout : float
            Seconds for network timeout.
        """
        self._token   = token
        self._timeout = timeout
        self._head    = {"Digitraffic-User": app_id}
        if token:
            self._head["Authorization"] = f"Bearer {token}"

    # ── helpers ───────────────────────────────────────────────
    async def _get(self, path: str, **params):
        async with httpx.AsyncClient(timeout=self._timeout) as cx:
            r = await cx.get(f"{_FT}{path}", headers=self._head, params=params)
        r.raise_for_status()
        return r.json()

    async def _put(self, path: str, payload):
        if "Authorization" not in self._head:
            raise RuntimeError("This method requires a Bearer token")
        async with httpx.AsyncClient(timeout=self._timeout) as cx:
            r = await cx.put(f"{_FT}{path}", headers=self._head, json=payload)
        r.raise_for_status()
        return r.json()

    # ── public API ────────────────────────────────────────────
    async def facility(self, fid: int) -> Facility:
        return Facility(**await self._get(f"/facilities/{fid}"))

    async def facilities(self, **search) -> List[Facility]:
        js = await self._get("/facilities", **search)
        return [Facility(**f) for f in js["results"]]

    async def utilization(
        self,
        fid: int,
        *,
        capacity_type: str = "CAR",
        usage: str = "PARK_AND_RIDE",
    ) -> UtilRow:
        rows = [UtilRow(**u) for u in await self._get(f"/facilities/{fid}/utilization")]
        try:
            return next(r for r in rows if r.capacityType == capacity_type and r.usage == usage)
        except StopIteration:
            raise ValueError("No matching utilization row found")

    async def update_utilization(self, fid: int, rows: List[UtilRow]):
        """
        PUT one or more UtilRow objects for *your* facility.
        Requires the client to be initialized with a Bearer token.
        """
        payload = [r.model_dump(exclude_none=True) for r in rows]
        return [UtilRow(**u) for u in await self._put(f"/facilities/{fid}/utilization", payload)]

    # convenience: enums
    async def capacity_types(self) -> EnumList:
        return EnumList(__root__=await self._get("/capacity-types"))

    async def usages(self) -> EnumList:
        return EnumList(__root__=await self._get("/usages"))


In [ ]:
#|export
# Default facility id for Urheilupuisto / Metro Areena (May-2025 feed)
METRO_AREENA_ID = 285        # use facilities() to discover others

async def parking_status(
    facility_id: int = METRO_AREENA_ID,
    *,
    capacity_type: str = "CAR",
    usage: str = "PARK_AND_RIDE",
    client: FintrafficParking | None = None,
) -> dict:
    """
    **Live Park-and-Ride vacancy** for a single facility.

    Parameters
    ----------
    facility_id    : int
        LIIPI/Fintraffic numeric id (default = Urheilupuisto P+R).
    capacity_type  : str
        "CAR", "ELECTRIC_CAR", "BICYCLE", …  (see `/capacity-types`).
    usage          : str
        "PARK_AND_RIDE" (default) or "COMMERCIAL".
    client         : FintrafficParking | None
        Re-use an existing client if you have one (saves a TCP handshake).

    Returns
    -------
    dict
        {
          "capacity"        : 320,
          "spacesAvailable" : 147,
          "timestamp_utc"   : "2025-05-20T15:56:03Z",
          "openNow"         : True
        }
    """
    api = client or FintrafficParking(app_id="datatalks-bot")
    row = await api.utilization(
        facility_id,
        capacity_type=capacity_type,
        usage=usage,
    )
    ts_utc = (
        dt.datetime.fromisoformat(row.timestamp)
        .astimezone(dt.timezone.utc)
        .isoformat(timespec="seconds")
    )
    return {
        "capacity":        row.capacity,
        "spacesAvailable": row.spacesAvailable,
        "timestamp_utc":   ts_utc,
        "openNow":         row.openNow,
    }


In [ ]:
#|export
async def osrm_walk(src, dst):
    "Lightweight wrapper: walking route via OSRM profile `foot`."
    return await osrm_car(src, dst, profile="foot")   # delegating call
